# extract FITS image for LSSTCam   

In [2]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry


In [3]:
repo = 'embargo'
instrument = 'LSSTCam'
datasetType = 'post_isr_image'
#datasetType = 'preliminary_visit_image'  # background subtracted calibrated image that detection has run on

collections = ["LSSTCam/runs/nightlyValidation", "LSSTCam/defaults",]

butler = Butler(repo, collections=collections, instrument=instrument)

registry = butler.registry


#uncomment to get the help page
#help(registry) 

## Find the exposures

BLOCK-T519 is the LSSTCam Local Meridian Observations at 80 deg elevation on day = 20250524

BLOCK-T517 — LSSTCam “Sidereal drive off tests” with and without sidereal tracking on day = 20250522

May 3 had over 500 science quality images of M49, Prawn, Trifid-Lagoon,  day_obs = 20250503, 
BLOCK-T365 denotes sciene image taking .


In [16]:

day = 20250522
day2 = 20250524

block = 'BLOCK-T519'  
dims = ['exposure']
datasetType = 'post_isr_image'

result = registry.queryDataIds( datasets=datasetType, dimensions = dims, where = 
          f"(day_obs = {day} OR day_obs = {day2})   AND exposure.science_program='{block}' "

                             )
meridianresults = [r for r in result]

print (f"got {len(meridianresults)} images")

got 30316 images


In [18]:

day = 20250522
block = 'BLOCK-T517'  

nonsiderealresult = registry.queryDataIds( datasets=datasetType, dimensions = dims, where = 
          f"day_obs = {day}   AND exposure.science_program='{block}' "

                             )
nonsiderealresult = [r for r in result]

print (f"got {len(nonsiderealresult)} images")

got 30316 images


In [ ]:

day = 20250503
block = 'BLOCK-T519'  #  AND exposure.science_program='{block}' - not workgin
target = 'M49'  # change to Prawn etc. to get others
dims = ['exposure']
datasetType = 'post_isr_image'

result = registry.queryDataIds( datasets=datasetType, dimensions = dims, where = 
          f"day_obs = {day}   AND exposure.target_name='{target}' "
                             )
results = [r for r in result]

#print(r['exposure']

print (f"got {len(results)} images")

LSSTCam has 202 images per exposure - this will list the exposures. 
The last digits after the daynumber  match the 'Seq. No.' in RubinTV

In [19]:
exp=""
exposures = []
for r in nonsiderealresult:
   if r['exposure'] != exp:
        exp=  r['exposure']
        exposures.append(exp)

print (f"there are {len(exposures)} exposures")
print (exposures)


there are 163 exposures
[2025052200226, 2025052200236, 2025052200237, 2025052200238, 2025052200239, 2025052200240, 2025052200241, 2025052200242, 2025052200243, 2025052200244, 2025052200245, 2025052200246, 2025052200247, 2025052200248, 2025052200249, 2025052200250, 2025052200251, 2025052200252, 2025052200253, 2025052200254, 2025052200255, 2025052200256, 2025052200257, 2025052200258, 2025052200259, 2025052200260, 2025052200261, 2025052200262, 2025052200263, 2025052200264, 2025052200265, 2025052200266, 2025052200267, 2025052200268, 2025052200269, 2025052200270, 2025052200271, 2025052200272, 2025052200273, 2025052200274, 2025052200275, 2025052200276, 2025052200283, 2025052200284, 2025052200285, 2025052200286, 2025052200287, 2025052200288, 2025052200289, 2025052200290, 2025052200291, 2025052200292, 2025052200293, 2025052200294, 2025052200295, 2025052200296, 2025052200297, 2025052200298, 2025052200299, 2025052200300, 2025052200301, 2025052200302, 2025052200303, 2025052200304, 2025052200305, 

This is just getting the datasetref for an exposure (this is LSSTcam so that should be 189 images one per detector) unless the corner rafts show up

In [ ]:
datasetRefs = registry.queryDatasets(datasetType=datasetType,
                                     where=f'exposure = {exposures[100]}')


## Write fits 
Write out fits files for the first count images. 
By default you will not have enough space to write all the files.

In [ ]:
images = []
count = 0 
stop = 2  # just write this many images
for count, exp in enumerate(datasetRefs):
    fn=f"Rubin-{exp.dataId['exposure']}-{exp.dataId['detector']}.fits"
    img = butler.get(datasetType,exp.dataId)
    img.writeFits(fn)
    images.append(exp)
    if count >= stop -1:
        break
    
print (f"{count+1} images output")

Now we have Fits files we can use with external code.
We get the list of files we made and process them with SEP (Source Extractor Python) as an example of external code. 
We will make a list of Objects and store that.

In [ ]:
import glob
filelist = glob.glob('Rubin-*.fits')
print (filelist)

## If you want to plot one of these 

In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

for exp in datasetRefs:
    dataId=exp.dataId
    img = butler.get('raw', dataId=dataId)

    fig = plt.figure()
    display = afwDisplay.Display()
    display.scale('linear', 'zscale')
    display.mtv(img)
    plt.show()